In [1]:
# import scikit-allel
import random
random.seed(42)
import time
import numpy as np
np.random.seed(42)
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import bcolz
import pandas
import allel; print('scikit-allel', allel.__version__)
# check which version is installed
print(allel.__version__)

callset = allel.read_vcf('/home/daron/bioInf/wilding/vcf_store/ag1000g.phase2.anopheles-rose.merged.biallelic.3.pruned.vcf.gz', fields='*')

# recode chrom pos in ndarray
pos = callset['variants/POS']
chrom = callset['variants/CHROM']
var = np.rec.fromarrays([chrom,pos], names='chrom,pos')

# create genotype array of whole dataset.
gt = allel.GenotypeArray(callset['calldata/GT'])
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

scikit-allel 1.3.2
1.3.2


In [2]:
# select subset of sample if needed
keepFile = pandas.read_csv("/home/daron/bioInf/wilding/ressources/k", sep='\t', header=None)

# coorespondance of samples order between snp and meta file
samples_list = list(callset["samples"])
samples_callset_index = [samples_list.index(s) for s in keepFile[0]]
# print(samples_callset_index)

# create a genotype array of the selected samples
gt = gt.take(samples_callset_index, axis=1)
print(gt.shape)

(1003463, 77, 2)


In [3]:
# remove variant with maf<0.01 and multi-allelic SNP and no segregating site
ac = gt.count_alleles(max_allele=3)
maf_cuttoff = 0.01
flt = ((ac.max_allele() == 1) & (1-ac.to_frequencies().max(axis=1) > maf_cuttoff) & ac.is_segregating()[:])
flt

gt_f = gt.compress(flt, axis=0)

gt_f


<GenotypeArray shape=(857565, 77, 2) dtype=int8>
0/1 0/0 0/0 0/0 0/0 ... 0/1 0/0 0/0 0/1 0/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/1
0/0 0/1 0/0 0/0 0/0 ... 0/0 0/0 0/1 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/1 0/0 0/0 0/0 0/0
0/1 0/1 0/0 0/0 0/1 ... 0/0 0/0 0/0 0/1 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [4]:
# Transform the genotypes into an array of alternate allele counts per call.
gn = gt_f.to_n_alt()[:]
coords, model = allel.pca(gn)
coords.shape


(77, 10)

In [7]:
# Print to file

# if subset 
coords_export = np.column_stack((callset["samples"][samples_callset_index], coords))

# else 
#coords_export = np.column_stack((callset["samples"], coords))

np.savetxt("/home/daron/bioInf/wilding/popstructure/pca/anopheles-rose.biallelic.3.pca.eigenvec.txt", coords_export, newline="\n", delimiter="\t", fmt='%s')
np.savetxt("/home/daron/bioInf/wilding/popstructure/pca/anopheles-rose.biallelic.3.pca.eigenval.txt", model.explained_variance_ratio_, newline="\n", delimiter="\t")

